1. get weekly resolution
2. get data in the box of interest
3. calculate stats
ezpz


In [1]:
import os
import xarray as xr
import geopandas as gpd

from tqdm import tqdm

import numpy as np
import pandas as pd

from joblib import Parallel, delayed

# Create masterfiles for each region
Each region will have a single netcdf file containing all daily data in the region. 

In [2]:
def expand_bounds(polygon, distance):
    return polygon.buffer(distance)

def load_polygons(polygon_filepath, buffer_distance=0):

    polygons  = gpd.GeoDataFrame.from_file(polygon_filepath)

    polygons['geometry'] = polygons['geometry'].apply(expand_bounds, distance=buffer_distance)
    return polygons


In [3]:
# Define the dictionary of groups
groups = {
    1: ['34', '00', '01'],
    2: ['02', '03'],
    3: ['04', '05', '06'],
    4: ['07', '08', '09', '10'],
    5: ['11', '12'],
    6: ['13', '14', '15'],
    7: ['16', '17', '18'],
    8: ['19', '20'],
    9: ['21', '22', '23'],
    10: ['24', '25'],
    11: ['26', '27', '28'],
    12: ['29', '30', '31'],
    13: ['32', '33']
}

# Iterate over each group in the dictionary
def process_group(group, regions):
    destination_foldername = f'chla_data_group_{group}'
    destination_path = os.path.join('/Volumes/Seagate 5TB/OceanColour Data/', 'regional_chla_data', destination_foldername)
    files = [os.path.join(destination_path, f) for f in os.listdir(destination_path) if f.endswith('.nc')]
    # files.sort()
    files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    # files[:5]
    # Load the polygons
    polygon_folder  = '/Users/tara/Documents/SJSU/MLML/polygons/pan_gl_regions/coastal_regions'
    polygon_file    = 'gl_coastal_regions.shp'

    polygon_filepath = os.path.join(polygon_folder, polygon_file)

    # expand the bounds of a polygon
    buffer_distance = 0.1

    polygons = load_polygons(polygon_filepath, buffer_distance)
    polygons = polygons[polygons['Region'].isin(regions)]
    regions = polygons['Region'].unique()
    
    polygon_bounds = {}
    for region in regions:
        polygon = polygons.loc[polygons['Region'] == region, 'geometry'].values[0]
        minx, miny, maxx, maxy = polygon.bounds
        polygon_bounds[region] = (minx, miny, maxx, maxy)

    
    for region in regions:
        minx, miny, maxx, maxy = polygon_bounds[region]

        output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_masterfiles/'
        if not os.path.exists(output_fp):
            os.makedirs(output_fp)
        if os.path.exists(os.path.join(output_fp, f'ds_masterfile_region_{region}.nc')):
            print(f"File already exists for region {region}. Skipping...")
            continue

        print("Processing Region: ", region)
        
        # Create an empty list to store the datasets
        datasets = []

        # Iterate over each file
        num_files = len(files)
        for i, file in enumerate(tqdm(files, desc="Processing files")):
            # Open the file as an xarray dataset
            ds = xr.open_dataset(file)
            ds = ds.where((ds.lon >= minx) & (ds.lon <= maxx) & (ds.lat >= miny) & (ds.lat <= maxy), drop=True)
            ds = ds.where((ds >= 0) & (ds <= 100)) # filter out negative and extreme values
            # Append the dataset to the list
            datasets.append(ds)
            ds.close()

        # Concatenate all the datasets along the time dimension
        print("Concatenating datasets...\n")
        ds_all = xr.concat(datasets, dim='time')
        del datasets

        ds_all = ds_all.where((ds_all.lon >= minx) & (ds_all.lon <= maxx) & (ds_all.lat >= miny) & (ds_all.lat <= maxy), drop=True)
        ds_all = ds_all.where((ds_all >= 0) & (ds_all <= 100)) # filter out negative and super extreme values

        # save ds_all as a .nc file
        output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_masterfiles/'
        if not os.path.exists(output_fp):
            os.makedirs(output_fp)
        ds_all.to_netcdf(os.path.join(output_fp, f'ds_masterfile_region_{region}.nc'))
        ds_all.close()


for group, regions in groups.items():
    process_group(group, regions)

File already exists for region 00. Skipping...
File already exists for region 01. Skipping...
File already exists for region 34. Skipping...
File already exists for region 02. Skipping...
File already exists for region 03. Skipping...
File already exists for region 04. Skipping...
File already exists for region 05. Skipping...
File already exists for region 06. Skipping...
File already exists for region 07. Skipping...
File already exists for region 08. Skipping...
File already exists for region 09. Skipping...
File already exists for region 10. Skipping...
File already exists for region 11. Skipping...
File already exists for region 12. Skipping...
Processing Region:  13


Processing files: 100%|██████████| 192/192 [09:15<00:00,  2.89s/it]


Concatenating datasets...

Processing Region:  14


Processing files: 100%|██████████| 192/192 [09:51<00:00,  3.08s/it]


Concatenating datasets...

Processing Region:  15


...

Concatenating datasets...

Processing Region:  32


Processing files: 100%|██████████| 192/192 [03:01<00:00,  1.06it/s]


Concatenating datasets...

Processing Region:  33


Processing files: 100%|██████████| 192/192 [02:59<00:00,  1.07it/s]


Concatenating datasets...



# Create weekly resolution files
10x smaller file size to work with

In [6]:
output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_masterfiles/'

In [13]:
def weekly_average(ds):
    ds['time'] = pd.to_datetime(ds['time'], origin='unix', unit='D')
    
    # Resample to weekly data and compute the mean
    weekly_ds = ds.resample(time='W').mean()

    return weekly_ds

def run_resample_data(region):

    master_files_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_masterfiles/'
    ds_all = xr.open_dataset(os.path.join(master_files_fp, f'ds_masterfile_region_{str(region).zfill(2)}.nc'))

    weekly_ds_all = weekly_average(ds_all)
    ds_all.close()

    # write weekly_ds_all to a netcdf file
    output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_weekly_files/'
    if not os.path.exists(output_fp):
        os.makedirs(output_fp)

    weekly_ds_all.to_netcdf(os.path.join(output_fp, f'ds_weekly_file_region_{str(region).zfill(2)}.nc'))
    weekly_ds_all.close()

for region in range(0, 35):
    print(f"Resampling data for region {region}...")
    run_resample_data(region)


Resampling data for region 0...
Resampling data for region 1...
Resampling data for region 2...
Resampling data for region 3...
Resampling data for region 4...
Resampling data for region 5...
Resampling data for region 6...
Resampling data for region 7...
Resampling data for region 8...
Resampling data for region 9...
Resampling data for region 10...
Resampling data for region 11...
Resampling data for region 12...
Resampling data for region 13...
Resampling data for region 14...
Resampling data for region 15...
Resampling data for region 16...
Resampling data for region 17...
Resampling data for region 18...
Resampling data for region 19...
Resampling data for region 20...
Resampling data for region 21...
Resampling data for region 22...
Resampling data for region 23...
Resampling data for region 24...
Resampling data for region 25...
Resampling data for region 26...
Resampling data for region 27...
Resampling data for region 28...
Resampling data for region 29...
Resampling data for 

# Get some stats
trendline, std error, r**2

In [2]:
def compute_row(lat_idx, ds_all):
    results = []
    for lon_idx in range(len(ds_all.lon)):
        y = ds_all['chlor_a'].isel(lat=lat_idx, lon=lon_idx)
        valid_mask = np.isfinite(y)
        # x = x_ # independent variable: time
        x = np.arange(len(ds_all.time))
        if np.unique(y[valid_mask]).size > 2: # at least 2 unique values needed to compute a slope
            numpts_ = np.unique(x[valid_mask]).size
            (slope, intercept), cov = np.polyfit(x[valid_mask], y[valid_mask], 1, cov=True)
            stderr = np.sqrt(np.nanmean(np.diag(cov)))

            trendline_values = np.polyval((slope, intercept), x)
            
            correlation_matrix = np.corrcoef(y[valid_mask], trendline_values[valid_mask])
            r_squared_value = correlation_matrix[0, 1] ** 2

            results.append((lat_idx, lon_idx, numpts_, slope, intercept, stderr, trendline_values, r_squared_value))
        else: 
            results.append((lat_idx, lon_idx, 0, np.nan, np.nan, np.nan, np.nan, np.nan))
    return results

def save_stats(region, results, ds_all):
    # results.append((lat_idx, lon_idx, numpts_, slope, intercept, stderr, trendline_values, r_squared_value))
    # lat_idx, lon_idx, numpts, slope, intercept, stderr, trendline, r_squared = zip(*results)
    numpts = np.full_like(ds_all['chlor_a'].isel(time=0), 0) # How many valid points (days) were used to compute the slope for each pixel. use this to fix the slope
    slope = np.full((len(ds_all.lat), len(ds_all.lon)), np.nan)

    stderr = np.full((len(ds_all.lat), len(ds_all.lon)), np.nan)
    r_squared = np.full((len(ds_all.lat), len(ds_all.lon)), np.nan)

    trendline = np.full((len(ds_all.time), len(ds_all.lat), len(ds_all.lon)), np.nan)


    # Flatten the list of lists
    results = [item for sublist in results for item in sublist]

    # Assign the results to the arrays
    for result in results:
        if result is not None:
            lat_idx, lon_idx, numpts_, slope_, intercept, stderr_, trendline_values, r_squared_value = result
            numpts[lat_idx, lon_idx] = numpts_
            slope[lat_idx, lon_idx] = slope_
            stderr[lat_idx, lon_idx] = stderr_
            trendline[:, lat_idx, lon_idx] = trendline_values
            r_squared[lat_idx, lon_idx] = r_squared_value
    del results

    print("Creating xarray dataset...")
    trendline_ds = xr.Dataset({'trendline': (('time', 'lat', 'lon'), trendline),
                            'r_squared': (('lat', 'lon'), r_squared),
                            'slope': (('lat', 'lon'), slope),
                            'numpts': (('lat', 'lon'), numpts),
                            'stderr': (('lat', 'lon'), stderr)},
                            coords={'lat': ds_all.lat,
                                    'lon': ds_all.lon,
                                    'time': ds_all.time})

    # Save the dataset to a netCDF file
    print("Saving results...")
    # output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/test5'
    output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_weekly_stats/'
    if not os.path.exists(output_fp):
        os.makedirs(output_fp)

    trendline_ds.to_netcdf(os.path.join(output_fp, f'trendline_weekly_region_{str(region).zfill(2)}.nc'))

for region in range(0, 35):
    output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_weekly_stats/'
    output_file = os.path.join(output_fp, f'trendline_weekly_region_{str(region).zfill(2)}.nc')
    if os.path.exists(output_file):
        print(f"File already exists for region {region}. Skipping...")
        continue
    print("Working on region: ", region)
    weekly_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/regional_weekly_files/'
    ds = xr.open_dataset(os.path.join(weekly_fp, f'ds_weekly_file_region_{str(region).zfill(2)}.nc'))
    results = Parallel(n_jobs=-1)(delayed(compute_row)(lat_idx, ds) for lat_idx in tqdm(range(len(ds.lat)), desc="Processing rows"))
    save_stats(region, results, ds)
    ds.close()
    del results


File already exists for region 0. Skipping...
File already exists for region 1. Skipping...
File already exists for region 2. Skipping...
File already exists for region 3. Skipping...
File already exists for region 4. Skipping...
File already exists for region 5. Skipping...
File already exists for region 6. Skipping...
File already exists for region 7. Skipping...
File already exists for region 8. Skipping...
File already exists for region 9. Skipping...
File already exists for region 10. Skipping...
File already exists for region 11. Skipping...
File already exists for region 12. Skipping...
File already exists for region 13. Skipping...
File already exists for region 14. Skipping...
File already exists for region 15. Skipping...
File already exists for region 16. Skipping...
File already exists for region 17. Skipping...
File already exists for region 18. Skipping...
File already exists for region 19. Skipping...
File already exists for region 20. Skipping...
File already exists for

Processing rows: 100%|██████████| 221/221 [16:11<00:00,  4.40s/it]


Creating xarray dataset...
Saving results...
Working on region:  31


Processing rows: 100%|██████████| 210/210 [19:36<00:00,  5.60s/it]


Creating xarray dataset...
Saving results...
Working on region:  32


...

Creating xarray dataset...
Saving results...
Working on region:  33


Processing rows: 100%|██████████| 195/195 [13:02<00:00,  4.01s/it]


Creating xarray dataset...
Saving results...
Working on region:  34


Processing rows: 100%|██████████| 107/107 [02:50<00:00,  1.59s/it]


Creating xarray dataset...
Saving results...


In [ ]:
# # log: 04/21/2023 224m to run 
# ds_all = xr.open_dataset(os.path.join(output_fp, f'ds_masterfile_region_{region}'))
# def compute_row(lat_idx):
#     results = []
#     for lon_idx in range(len(ds_all.lon)):
#         y = ds_all['chlor_a'].isel(lat=lat_idx, lon=lon_idx)
#         valid_mask = np.isfinite(y)
#         x = x_ # independent variable: time
#         if np.unique(y[valid_mask]).size > 2: # at least 2 unique values needed to compute a slope
#             numpts_ = np.unique(x[valid_mask]).size
#             (slope, intercept), cov = np.polyfit(x[valid_mask], y[valid_mask], 1, cov=True)
#             stderr = np.sqrt(np.nanmean(np.diag(cov)))

#             trendline_values = np.polyval((slope, intercept), x)
            
#             correlation_matrix = np.corrcoef(y[valid_mask], trendline_values[valid_mask])
#             r_squared_value = correlation_matrix[0, 1] ** 2

#             results.append((lat_idx, lon_idx, numpts_, slope, intercept, stderr, trendline_values, r_squared_value))
#         else: 
#             results.append((lat_idx, lon_idx, 0, np.nan, np.nan, np.nan, np.nan, np.nan))
#     return results
    

# results = Parallel(n_jobs=-1)(delayed(compute_row)(lat_idx) for lat_idx in tqdm(range(len(ds_all.lat)), desc="Processing rows"))

In [ ]:
# print("Creating xarray dataset...")
# trendline_ds = xr.Dataset({'trendline': (('time', 'lat', 'lon'), trendline),
#                            'r_squared': (('lat', 'lon'), r_squared),
#                            'slope': (('lat', 'lon'), slope),
#                            'numpts': (('lat', 'lon'), numpts),
#                            'stderr': (('lat', 'lon'), stderr)},
#                           coords={'lat': ds_all.lat,
#                                   'lon': ds_all.lon,
#                                   'time': ds_all.time})

# # Save the dataset to a netCDF file
# print("Saving results...")
# output_fp = '/Volumes/Seagate 5TB/OceanColour Data/statistics/test5'
# if not os.path.exists(output_fp):
#     os.makedirs(output_fp)

# trendline_ds.to_netcdf(os.path.join(output_fp, 'trendline.nc'))